<a href="https://colab.research.google.com/github/JasonHyun/GenderRecall/blob/main/CS4701.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Authors: \\
Jacobo Carreon (jic56) \\
Jason Hyun (jh986) \\
Amanda Ni (kn297)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Project Summary
Problem with Google Translate to Spanish: **Gender Recall** \\
What is Gender Recall? Gender of an object is not remembered from one sentence to the next. \\
Why is this important? Spanish is the second most spoken language in the United States amd accurate translations are essential for communication. 

Since this project is specifically designed to propose a solution to a singular issue the following restrictions will hold: \\
1. The first sentence will contain the noun whose gender is being predicted. The first sentence will be restricted to ***only*** that noun. 
2. The rest of the sentences will contain adjectives describing the noun. Spanish categorizes nouns as masculine or feminine. Masculine nouns for the most part are described with adjectives ending in "o" or "os" if plural, or "a" or "as" if feminine. 

Example: English Sentence: "The pigeon (noun) walked around the block. It flew away because of how scared (adj) it was. " \\
Google's translation: "La paloma caminaba alrededor de la cuadra. Voló por lo **asustado** que estaba" \\
Correct Translation: "La paloma caminaba alrededor de la cuadra. Voló por lo **asustada** que estaba"

What will we show? Implementing a Long Short Term Memory (LSTM) Network to keep track of the gender of noun will create a more accurate translation than the current implementation. 



In [ ]:
#imports
!pip install --user -U nltk
!pip install googletrans==4.0.0-rc1
!pip install datasets
import nltk
import googletrans
nltk.download('punkt')
!pip install -U spacy
import spacy
from spacy import displacy
from collections import Counter
import pandas as pd
pd.set_option("max_rows", 400)
pd.set_option("max_colwidth", 400)
!python -m spacy download es_core_news_md
# from datasets import load_from_disk

# raw_train = load_from_disk("test.hf")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 4.5 MB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.8 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 1.5 MB 18.4 MB/s 
     |████████████████████████████████| 65 kB 4.0 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17414 sha256=4f83fe6e8e52620513224ed5f516398084598af63bd9239ba7bc22daaaa2b8b5
  Stored in directory: /root/.cache/pip/wheels/a9/25/af/715361fa79594524c89c75d293def652045f3fdce6ca398712
Successfully built googletrans
Looking in indexes: https://pypi.org/simple, h

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-14 14:56:02.209944: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.3 MB 1.2 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


In [ ]:
#sentences that are translated incorrectly

sentenceOne = "The pigeon walked around the block the dog. It flew away because of how scared it was. "

sentenceTwo = "We are having a girls night tonight. We are so excited."

In [ ]:
# shortened_raw_train = raw_train['train'][:45000]

In [ ]:
# train = []
# for sentence in shortened_raw_train['text']:
#   if(len(sentence) > 40):
#     train.append(word_tokenize(sentence))
# print(train)

In [ ]:
#Source: https://pypi.org/project/googletrans/
#Returns a string representation of the spanish translation of an english sentence 

from googletrans import Translator

translator = Translator()

def translate(english_sentence):

  translated_sentence = translator.translate(english_sentence, dest='es').text

  lst = translated_sentence.split(".")
  if(len(lst) >= 3):
    fixed_translated = lst[0]
    for x in lst[1:-1]:
      fixed_translated += (". " + x)
    return fixed_translated
  else:
    return translated_sentence



In [ ]:
#Returns a list of a sentence as tokens 

from nltk.tokenize import word_tokenize

def tokenize(sentence):

  return word_tokenize(sentence)



In [ ]:
#Source : https://melaniewalsh.github.io/Intro-Cultural-Analytics/05-Text-Analysis/Multilingual/Spanish/03-POS-Keywords-Spanish.html

#Returns a list of each spanish words part of speech 

nlp = spacy.load('es_core_news_md')

def spanishPos(spanish_sentence):

  posList = []

  for token in nlp(spanish_sentence): 

    posList.append(token.pos_)

  return posList

spanishPos("correr")


['VERB']

In [ ]:
from nltk.classify.positivenaivebayes import PositiveNaiveBayesClassifier

#Returns a string list of the gender of each adjective found. M for male and F for female. 

def adjectiveGender(spanishWord):
  pos = spanishPos(spanishWord)[0]
  # adjList = []
  # for index in range(len(spanish_pos_list)):
  #   try:
  #     word = tokenized_spanish_sentence[index]
  #     word_pos = spanish_pos_list[index]
  #   except:
  #     return adjList
  female_pronouns = ["ella", "ellas", "Ella", "Ellas", "Nosotras", "nosotras", "vosotras", "Vosotras"]
  male_pronouns = ["el", "ellos", "El", "Ellos", "Nosotros", "nosotros", "vosotros", "Vosotros"]

  if(pos == "DET"):
      if(spanishWord[-1] == 'a'):
        return "F-DET" 
      elif(spanishWord[-1] == "s" and spanishWord[-2] == "a"):
        return "F-DET"
      else: 
        return "M-DET" 
    
  elif(pos == "PRON"):
      if(spanishWord in female_pronouns):
        return "F-PRON"
      # elif(spanishWord in male_pronouns):
      #   return "O"
      else:
        return "M-PRON"
  elif(pos == "NOUN"):
    return "NOUN"
        

  elif(pos == "ADJ"):
      #plural adjectives
      if(spanishWord[-1] == 's'):
        if (spanishWord[-2] == 'a'):
          return "F-ADJ" 
        else: return "M-ADJ"
      else:
        if(spanishWord[-1] == 'a'):
          return "F-ADJ"
        else: return "M-ADJ"
    # elif(spanishWord == "."):
    #   adjList.append("O")
  else:
      return "O"
  # return adjList
test = "Hello"
print(translate(test))
adjectiveGender(translate(test))

Hola


'O'

In [ ]:
#returns True if the gender Google Translate matches expected gender, otherwise False. 

# def predicted_gender(english_sentence, expected_gender):
#   translated_sentence = translate(english_sentence)
#   male_adjectives = 0
#   female_adjectives = 0
#   gender_predicted = ""
#   for x in adjectiveGender(translated_sentence):
#     if(x == "M"):
#       male_adjectives += 1
#     elif(x == "F"):
#       female_adjectives += 1
#   if(male_adjectives > female_adjectives):
#     gender_predicted = "M"
#   elif(male_adjectives < female_adjectives):
#     gender_predicted = "F"
#   return expected_gender == gender_predicted 

def blindAdj(spanishWord):
  pos = spanishPos(spanishWord)[0]
  if pos == "ADJ":
      if(spanishWord[-1] == 's' and spanishWord[-2] != 'e' ):
        spanishWord = spanishWord[0:-2] + "xs"
      elif(spanishWord[-1] != 'e'):
        spanishWord = spanishWord[0:-1] + "x"
  return spanishWord

blindAdj("enojado")
    





'enojadx'

In [ ]:
# train_data = dict()
# train_data['text'] = makeBlind(train['text'])
# train_data['tags'] = train['tags']
# train_data['index'] = train['index']
# # print(len(train))
# # print(len(train_tags))
# # print(len(train_index))

# with open(os.path.join(path, "train.json"), 'w') as json_file:
#     json.dump(train_data, json_file)


In [ ]:
import json
import math
import os
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')
path = os.path.join(os.getcwd(), "drive", "MyDrive", "cs4701 project")
with open(os.path.join(path,'train.json'), 'r') as f:
     train = json.loads(f.read())

def makeSet(name):
  with open(os.path.join(path, name), 'r') as f:
    text = ""
    for line in f:
        text += line
  text_list = text.split('.')
  final = []
  for sentence in text_list: 
    final.append(word_tokenize(sentence))
  return final
# testSet = makeSet('TrainingSetTijuana.txt')
validation = makeSet("TrainingSetC2.txt")
testSet = [""]
# validation = makeSet("ValidationSetMexico.txt")
# train = makeSet("TestSetIndigenous.txt")

def makeBlind(Set):
  blinded_set = []
  for sentence in Set:
    blinded_sentence = []
    for word in sentence:
      blinded_sentence.append(blindAdj(word))
    blinded_set.append(blinded_sentence)
  return blinded_set

testSetBlinded = testSet
validationBlinded = makeBlind(validation)


def makeIndex(text):
  count = 0
  index = []
  for sentence in text:
    for token in sentence:
      index.append(count +1)
      count+= 1
  return index

train_index = train['index']
val_index = makeIndex(validation)
test_index = makeIndex(testSet)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print(train['text'])

[['Somos', 'los', 'típicxs', 'amigos', 'de', 'la', 'secundarix', 'que', 'juraron', 'que', 'su', 'amistad', 'duraría', 'para', 'siempre', '.'], ['Aunque', 'las', 'circunstancias', 'del', 'inicio', 'de', 'nuestra', 'amistad', 'no', 'fueron', 'especialex', 'en', 'ciertx', 'modo', 'fueron', 'especialex', 'para', 'nosotros', '.'], ['[', 'Encuéntrate', 'conmigo', 'en', 'el', 'patio', 'de', 'atrás', '.', ']'], ['No', 'se', 'lo', 'acaricié', ',', 'me', 'quede', 'allí', 'parado', 'tocándolo', '.'], ['¿En', 'qué', 'estaban', 'pensando', 'para', 'pelear', 'así', 'por', 'una', 'chica', '?'], ['Ok', ',', 'solx', 'haremos', 'una', 'ronda', ',', 'como', 'hombres', '.'], ['Ya', 'que', 'gané', 'en', 'Piedra', ',', 'papel', 'o', 'tijeras', ',', 'y', 'ya', 'le', 'toqué', 'el', 'pecho', '.'], ['Vamos', ',', 'ni', 'que', 'nuestra', 'relación', 'fuera', 'a', 'durar', 'para', 'siempre', 'o', 'algo', '.'], ['Tan', 'solx', 'manténganse', 'cerca', 'mío', 'y', 'podrán', 'jugar', 'por', 'ella', 'cuando', 'rompamo

In [ ]:
def get_tags(set):

  tags = []
  for sentence in set:
    sentence_tag = []
    for word in sentence:
      sentence_tag.append(adjectiveGender(word))
    tags.append(sentence_tag)
  return tags
# train_tags = get_tags(train)
val_tags = get_tags(validation)

In [ ]:
test_sentences = [["The","quick","brown","fox","jumped","over","the","lazy","dog"],["The","quick","brown","fox","jumped"]]
pad_token_test = "<PAD>"


def longest_length(sents):
  sentence_lengths = [len(sentence) for sentence in sents]
  longest_sentence_length = max(sentence_lengths)
  return longest_sentence_length
#changed to training

max_len = longest_length(train['text'])
print(longest_length(train['text']))
def pad_sents(sents, pad_token, max_len):
    """ Pad list of sentences according to the longest sentence in the batch.
        The paddings should be at the end of each sentence.
    
    Arguments:
      sents(list[list[str]]): list of sentences, where each sentence
                                    is represented as a list of words
      pad_token(str): padding token

      max_len(int): length of the longest sentence in the dataset

    Returns:
      sents_padded(list[list[str]]): list of sentences where sentences shorter
        than the max length sentence are padded out with the pad_token, such that
        each sentence in the batch now has equal length.
    """
    ### TO DO: Write a function that will pad the sentences according to the longest sentence in the training data ###

    #"The quick brown fox jumped over the lazy dog"
    #"The quick brown fox jumped <PAD> <PAD> <PAD> <PAD>"


    sentence_lengths = [len(sentence) for sentence in sents]
    longest_sentence_length = max_len
    padded_sentences =[]

    for sent in sents:
      padded_sentences.append(list(sent) + [pad_token]*(longest_sentence_length - len(sent)))

    return padded_sentences

#test
pad_sents(train['text'], "<>", 55)

70


[['Somos',
  'los',
  'típicxs',
  'amigos',
  'de',
  'la',
  'secundarix',
  'que',
  'juraron',
  'que',
  'su',
  'amistad',
  'duraría',
  'para',
  'siempre',
  '.',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>'],
 ['Aunque',
  'las',
  'circunstancias',
  'del',
  'inicio',
  'de',
  'nuestra',
  'amistad',
  'no',
  'fueron',
  'especialex',
  'en',
  'ciertx',
  'modo',
  'fueron',
  'especialex',
  'para',
  'nosotros',
  '.',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>',
  '<>'],
 ['['

In [ ]:
#FFNN was replicated by a project Jacobo created for CS 4740 but changed to fit gendered-adjectives tagging rather than Entity Tagging
from typing import List, Tuple, Dict, Set, Union
category_map = {"O": 0,
                "F-ADJ": 1,
                "F-PRON": 2, 
                "F-DET" : 3,
                "M-ADJ": 4, 
                "M-PRON": 5, 
                "M-DET" : 6,
                "NOUN": 7}

def encode_ner_category(category_data: List[List[str]])->List[List[int]]:
  """ Encoding NER category from a list of strings to a list of integers

  Arguments: 
    category_data (list(list(str))): NER categories

  Returns:
    encoded category (list(list(int))): Numerical conversions of NER categories
  """

  ### TODO : write a function that converts NER labels to integers ###
  
  result_sentences = []
  for sentence_NER in category_data:
    result_sentence = []
    for NER_label in sentence_NER:
     NER_label = category_map[NER_label]
     result_sentence.append(NER_label)
    result_sentences.append(result_sentence)


  return result_sentences


encode_ner_category(train['tags'])

[[0, 6, 4, 7, 0, 3, 1, 0, 0, 0, 6, 7, 0, 0, 0, 0],
 [0, 3, 7, 0, 7, 0, 3, 7, 0, 0, 4, 0, 4, 7, 0, 4, 0, 5, 0],
 [7, 0, 5, 0, 6, 7, 0, 0, 0, 7],
 [0, 5, 5, 0, 0, 5, 0, 0, 0, 0, 0],
 [0, 5, 0, 0, 0, 0, 0, 0, 3, 7, 0],
 [0, 0, 4, 0, 3, 7, 0, 0, 7, 0],
 [0, 0, 0, 0, 0, 0, 7, 0, 7, 0, 0, 0, 5, 0, 6, 7, 0],
 [0, 0, 0, 0, 3, 7, 0, 0, 0, 0, 0, 0, 5, 0],
 [0, 4, 0, 0, 5, 0, 0, 0, 0, 2, 0, 0, 0],
 [0, 0, 0, 6, 7, 0, 6, 0, 0, 5, 6, 7, 4, 0],
 [0, 7, 4, 0, 3, 7, 0],
 [0, 0, 7, 0, 4, 0, 3, 7, 0, 7, 0, 0, 0, 0],
 [0, 6, 7, 3, 0, 0, 1, 0, 0, 5, 5, 0, 4, 5, 0],
 [0, 0, 0, 6, 7, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 7, 0, 7, 0, 7, 0, 0, 5, 0, 5, 4, 0],
 [0, 3, 7, 0, 3, 0, 0, 0, 3, 7, 0, 3, 1, 0, 3, 7, 0],
 [0, 0, 0, 3, 7, 0, 0, 4, 0, 0],
 [0, 0, 0, 5, 0, 6, 7, 0, 0, 3, 0, 0],
 [5, 0, 0, 0, 4, 0, 5, 0],
 [6,
  7,
  0,
  3,
  7,
  0,
  0,
  4,
  0,
  5,
  6,
  0,
  5,
  0,
  4,
  0,
  3,
  0,
  0,
  3,
  7,
  0,
  5,
  0,
  7,
  0,
  0,
  0,
  0,
  6,
  4,
  0,
  6,
  0,
  0],
 [0, 0, 0, 0, 6, 0, 0, 6, 7, 0, 

In [ ]:
UNK = "<UNK>"
PAD = "<PAD>"
from itertools import chain
class Vocab(object):
    """ Vocabulary, i.e. structure containing language terms.

      Instance attributes:
        word2id: dictionary mapping words to indices
        unk_id: index for UNK
        id2word: dictionary mapping indices to words

    """
    def __init__(self, word2id=None):
        """ Init Vocab Instance.
        
        :param word2id: dictionary mapping words 2 indices
        :type word2id: dict[str, int]
        """
        if word2id:
            self.word2id = word2id
        else:
            self.word2id = dict()
            self.word2id[PAD] = 0   # Pad Token
            self.word2id[UNK] = 1   # Unknown Token
        self.unk_id = self.word2id[UNK]
        self.id2word = {v: k for k, v in self.word2id.items()}

    def __getitem__(self, word):
        """ Retrieve word's index. Return the index for the unk
        token if the word is out of vocabulary.
        
        :param word: word to look up
        :type word: str
        :returns: index of word
        :rtype: int
        """
        return self.word2id.get(word, self.unk_id)

    def __contains__(self, word):
        """ Check if word is captured by Vocab.
        
        :param word: word to look up
        :type word: str
        :returns: whether word is in vocab
        :rtype: bool
        """
        return word in self.word2id

    def __setitem__(self, key, value):
        """ Raise error, if one tries to edit the Vocab directly.
        """
        raise ValueError('vocabulary is readonly')

    def __len__(self):
        """ Compute number of words in Voca.
        
        :returns: number of words in Vocab
        :rtype: int
        """
        return len(self.word2id)

    def __repr__(self):
        """ Representation of Vocab to be used
        when printing the object.
        """
        return 'Vocabulary[size=%d]' % len(self)

    def word_from_id(self, wid):
        """ Return mapping of index to word.
        
        :param wid: word index
        :type wid: int
        :returns: word corresponding to index
        :rtype: str
        """
        return self.id2word[wid]

    def add(self, word):
        """ Add word to Vocab, if it is previously unseen.
        
        :param word: to add to Vocab
        :type word: str
        :returns: index that the word has been assigned
        :rtype: int
        """
        if word not in self:
            wid = self.word2id[word] = len(self)
            self.id2word[wid] = word
            return wid
        else:
            return self[word]

    ############################################################################
    #######################            CHANGED            ######################
    ############################################################################
    @staticmethod
    def from_corpus(corpus, remove_frac=None, freq_cutoff=None): 
        """ Given a corpus construct a Vocab.
        
        :param corpus: corpus of text produced by read_corpus function
        :type corpus: List[str]
        :param freq_cutoff: if word occurs n < freq_cutoff times, drop the word
        :type freq_cutoff: int
        :returns: Vocab instance produced from provided corpus
        :rtype: Vocab
        """
        vocab_entry = Vocab()
        word_freq = Counter(chain(*corpus))
        if freq_cutoff is None:
            freq_cutoff = 1
        valid_words = [w for w, v in word_freq.items() if v >= freq_cutoff]
        print('number of word types: {}, number of word types w/ frequency >= {}: {}'
              .format(len(word_freq), freq_cutoff, len(valid_words)))
        top_words = sorted(valid_words, key=lambda word: word_freq[word], reverse=True)
        if remove_frac is not None:
            size = len(top_words) - int(remove_frac * len(top_words))
            top_words = top_words[:size]
            print(f'number of unique words retained with remove_frac={remove_frac}: {len(top_words)}')
        for word in top_words:
            vocab_entry.add(word)
        return vocab_entry

In [ ]:
# Setting seed ***DO NOT MODIFY***
import torch
torch.manual_seed(123)

################################################################################
#########################            CHANGED            ########################
################################################################################
print('initialize train vocabulary ..')
vocab = Vocab.from_corpus(train['text'], remove_frac=0.5)

initialize train vocabulary ..
number of word types: 10751, number of word types w/ frequency >= 1: 10751
number of unique words retained with remove_frac=0.5: 5376


In [ ]:
def get_masks(sents_padded, pad_token):
    """ Create masks for a list of sentences according to the longest sentence 
        in the training data.
        The mask should be a binary array.
    
    Arguments:
      sents_padded(list[list[str]]): list of sentences where sentences shorter
        than the max length sentence are padded out with the pad_token, such that
        each sentence in the batch now has equal length.

      pad_token(str): padding token

    Returns:
      sents_masks(list[list[Int]]): list of 0/1 where words shorter
        than the max length in the sentence are denoted as 1 and 0 otherwise.
    """
    all_masks = []
    for sent in sents_padded:
      all_masks.append([0 if w == pad_token else 1 for w in sent])
    return all_masks

In [ ]:
from torch.utils.data import Dataset, DataLoader, Subset
class LanguageDataset(Dataset):
    """LanguageDataset is a torch dataset to interact with the Language data.

        Dataset (List[Tuple[List[torch.Tensor], int]]):  The vectorized dataset with input and expected output values.
        Dataset is an abstract class representing a dataset: https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset
        
    """
    def __init__(self, text, tag, mask):
        """
        Loads in the text and tags as tensors.

        Args:
            :param text: padded sentence (including padding)
            :type text: list(list(int))
            :param tag: encoded labels (including padding)
            :type tag: list(list(int): 
            :param mask: mask for the padded sentences (including padding)
            :type tag: list(list(int): 
        """
        self.text = torch.tensor(text)
        self.tag = torch.tensor(tag)
        self.mask = torch.tensor(mask, dtype=torch.bool)
        self.len = len(text)
    
    def __len__(self):
        """__len__ returns the number of samples in the dataset.

        :returns: number of samples in dataset
        :rtype: int
        """
        return self.len

    def __getitem__(self, index):
        """__getitem__ returns the tensor, output pair for a given index

        :param index: index within dataset to return
        :type index: int
        :returns: A tuple (x, y, m) where x is model input, y is our label and m is the mask
        :rtype: Tuple[torch.Tensor, torch.Tensor, torch.Tensor]
        """

        return self.text[index], self.tag[index], self.mask[index]

In [ ]:
def get_data_loaders(preprocessed_data, batch_size=1, shuffle=False):
    """Pads, vectorizes, and batchifies the dataset.

    Args:
        preprocessed_data (dict(str:list(list(str), str:list(list(int))))): The processed data with 'text', 'NER', 'mask' and 'text-tokenized' labels
    """
    dataset = LanguageDataset(preprocessed_data['text-tokenized'], preprocessed_data['NER'], preprocessed_data['mask'])
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return loader

In [ ]:
def tokenize_sentences(sentences, vocab):
    output = []
    for sentence in sentences:
        tokenized_sentence = [vocab[w] for w in sentence]
        output.append(tokenized_sentence)
    return output

In [ ]:


init_keys = {'NER': None, 'text': None}

processed_train = dict()
processed_val = dict()

processed_train.update(init_keys)
processed_val.update(init_keys)

print(longest_length(train['text']))

processed_train['NER'] = pad_sents(encode_ner_category(train['tags']), 0, longest_length(train['text']))
processed_train['text'] = pad_sents(train['text'], '<PAD>', longest_length(train['text']))

processed_train['text-tokenized'] = tokenize_sentences(processed_train['text'], vocab)
processed_train['mask'] = get_masks(processed_train['text'], PAD)

#processed_val['NER'] = encode_ner_category(processed_val['NER'])

processed_val['NER'] = pad_sents(encode_ner_category(val_tags), 0, longest_length(train['text']))
processed_val['text'] = pad_sents(validationBlinded, '<PAD>', longest_length(train['text']))


processed_val['text-tokenized'] = tokenize_sentences(processed_val['text'], vocab)
processed_val['mask'] = get_masks(processed_val['text'], PAD)

# print(longest_length(train_tags))
# print(len(processed_val['text'][0]))
# print(processed_val)


processed_test = dict()
processed_test['NER'] =  pad_sents(encode_ner_category(train['tags']), 0, longest_length(train['text']))
processed_test['text'] = pad_sents(testSet, '<PAD>', longest_length(train['text']))
processed_test['text-tokenized'] = tokenize_sentences(processed_test['text'], vocab)
processed_test['mask'] = get_masks(processed_test['text'], PAD)

70


In [ ]:
torch.manual_seed(123)
batch_size = 64
#originally 64
train_loader = get_data_loaders(processed_train, batch_size=batch_size, shuffle=True)
val_loader = get_data_loaders(processed_val, batch_size=batch_size)
print(val_loader)
try:
  test_loader = get_data_loaders(processed_test, batch_size=batch_size)
except: 
  test_loader = []

In [ ]:
import torch
import torch.nn as nn
# Lambda to switch to GPU if available
get_device = lambda : "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Setting seed ***DO NOT MODIFY***
torch.manual_seed(123)

def weight_init(m):
	if isinstance(m, nn.Linear):
		nn.init.xavier_uniform_(m.weight)
		nn.init.constant_(m.bias, 0.)


class FFNN(nn.Module):
	def __init__(self, hidden_dim, embedding_dim, output_dim, vocab_size):
		super(FFNN, self).__init__()
		self.loss_class_weights = torch.tensor([0.5, 1, 1, 1, 1, 1, 1, 1, 1], 
		                                       dtype=torch.float)
		self.embedding = nn.Embedding(vocab_size, embedding_dim, max_norm=True)
		self.W1 = nn.Linear(embedding_dim, hidden_dim)            
		self.W2 = nn.Linear(hidden_dim, output_dim)
   

		self.relu = nn.ReLU()

		self.softmax = nn.LogSoftmax(dim=-1)
	

		self.loss = nn.NLLLoss()

	def compute_Loss(self, predicted_vector, gold_label, masks):
		return self.loss(predicted_vector[masks], gold_label[masks])

	def forward(self, input_vector):

		original_shape = input_vector.shape
		input_vector = input_vector.reshape(-1)
		

		embeddings = self.embedding(input_vector) 
		z1 = self.W1(embeddings)
		z1_relu = self.relu(z1)
		z2 = self.W2(z1_relu)  
	  
	
	

		predicted_vector = (z2)

	
	

		predicted_vector = predicted_vector.reshape((original_shape[0],
                                                 original_shape[1], -1))
	  
	
		return self.softmax(predicted_vector)

	def load_model(self, save_path):
		self.load_state_dict(torch.load(save_path))

	def save_model(self, save_path):
		torch.save(self.state_dict(), save_path)

In [ ]:

torch.manual_seed(123)

def train_epoch(model, train_loader, optimizer):
  model.train()
  total = 0
  batch = 0
  total_loss = 0
  correct = 0
  for (input_batch, expected_out, batch_mask) in tqdm(train_loader, leave=False, desc="Training Batches"):
    optimizer.zero_grad()
    batch += 1
    flattened_expected_out = expected_out.reshape(-1).to(device)
    flattened_batch_mask = batch_mask.reshape(-1).to(device)
    output = model(input_batch.to(get_device())).to(get_device())
    flattened_output = output.reshape(-1, output.shape[-1])
    loss = model.compute_Loss(flattened_output, flattened_expected_out, flattened_batch_mask)


    total += batch_mask.sum().item()
    _, predicted = torch.max(output, -1)
    flattened_predicted = predicted.reshape(-1)

    correct += (flattened_expected_out[flattened_batch_mask].to("cpu") == flattened_predicted[flattened_batch_mask].to("cpu")).cpu().numpy().sum()
    total_loss += loss.item()
    loss.backward()
    optimizer.step()
  print(total_loss)
    
  print("Loss: " + str(total_loss/batch))
  print("Training Accuracy: " + str(correct/total))
  return total_loss/batch

In [ ]:

torch.manual_seed(123)

from tqdm.notebook import tqdm, trange
import torch.optim as optim
def evaluation(model, val_loader, optimizer):
  model.eval()
  loss = 0
  correct = 0
  total = 0
  for (input_batch, expected_out, batch_mask) in tqdm(val_loader, leave=False, desc="Validation Batches"):
    flattened_expected_out = expected_out.reshape(-1).to(device)
    flattened_batch_mask = batch_mask.reshape(-1).to(device)
    output = model(input_batch.to(get_device())).to(get_device())
    flattened_output = output.reshape(-1, output.shape[-1])
    total += batch_mask.sum().item()
    _, predicted = torch.max(output, -1)
    flattened_predicted = predicted.reshape(-1)
    print(flattened_expected_out)
    correct += (flattened_expected_out[flattened_batch_mask].to("cpu") == flattened_predicted[flattened_batch_mask].to("cpu")).cpu().numpy().sum()
    loss += model.compute_Loss(flattened_output, flattened_expected_out, flattened_batch_mask)
    
  loss /= len(val_loader)
  print(loss)
  
  print("Validation Loss: " + str(loss.item()))
  print("Validation Accuracy: " + str(correct/total))
  print()
  return loss.item()

In [ ]:
torch.manual_seed(123)
def train_and_evaluate(number_of_epochs, model, train_loader, val_loader, min_loss=0, lr=.001):
  optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=.01)
  loss_values = [[],[]]
  for epoch in trange(number_of_epochs, desc="Epochs"):
    cur_loss = train_epoch(model, train_loader, optimizer)
    loss_values[0].append(cur_loss)
    cur_loss_val = evaluation(model, val_loader, optimizer)
    loss_values[1].append(cur_loss_val)
    if cur_loss <= min_loss: return loss_values
  return loss_values

In [ ]:

torch.manual_seed(123)

model = FFNN(max_len, 150, max_len, len(vocab))

model.apply(weight_init)



max_len_train = longest_length(train['text'])
num_NER_tags = 9
train_and_evaluate(30, model, train_loader, val_loader, .2, .0005)


Epochs:   0%|          | 0/30 [00:00<?, ?it/s]

Training Batches:   0%|          | 0/160 [00:00<?, ?it/s]

457.5034402012825
Loss: 2.859396501258016
Training Accuracy: 0.6922660502136126


Validation Batches:   0%|          | 0/3 [00:00<?, ?it/s]

tensor([0, 0, 0,  ..., 0, 0, 0])
tensor([0, 0, 0,  ..., 0, 0, 0])
tensor([0, 3, 7,  ..., 0, 0, 0])
tensor(1.0992, grad_fn=<DivBackward0>)
Validation Loss: 1.0991698503494263
Validation Accuracy: 0.7431192660550459



...

Training Batches:   0%|          | 0/160 [00:00<?, ?it/s]

25.05795142799616
Loss: 0.15661219642497598
Training Accuracy: 0.9624056300111196


Validation Batches:   0%|          | 0/3 [00:00<?, ?it/s]

tensor([0, 0, 0,  ..., 0, 0, 0])
tensor([0, 0, 0,  ..., 0, 0, 0])
tensor([0, 3, 7,  ..., 0, 0, 0])
tensor(0.3073, grad_fn=<DivBackward0>)
Validation Loss: 0.307254821062088
Validation Accuracy: 0.8711426188490409



[[2.859396501258016, 0.47864447273314, 0.15661219642497598],
 [1.0991698503494263, 0.40812528133392334, 0.307254821062088]]

In [ ]:
def makeSet2(text):
    text_list = text.split('.')
    final = []
    for sentence in text_list: 
      final.append(word_tokenize(sentence))
    return final[0:-1]
makeSet2("Hello, my name is Jacobo. I am thirty years old and happily married.")

[['Hello', ',', 'my', 'name', 'is', 'Jacobo'],
 ['I', 'am', 'thirty', 'years', 'old', 'and', 'happily', 'married']]

In [ ]:

def predict(text):
  Set = makeSet2(text)
  Set = makeBlind(Set)
  processed_test = dict()
  processed_test['NER'] =  pad_sents(encode_ner_category(train['tags']), 0, longest_length(train['text']))
  processed_test['text'] = pad_sents(Set, '<PAD>', longest_length(train['text']))
  processed_test['text-tokenized'] = tokenize_sentences(processed_test['text'], vocab)
  processed_test['mask'] = get_masks(processed_test['text'], PAD)
  loader = get_data_loaders(processed_test, batch_size=batch_size)
  index = makeIndex(Set)
  path = os.path.join(os.getcwd(), "drive", "MyDrive", "cs4701 project", "ffnn.model")

  model.save_model(path)
  loaded_model = FFNN(max_len, 150, max_len, len(vocab))
  loaded_model.load_model(path)


  flattened_output = []
  for (input_batch, expected_out, batch_mask) in tqdm(loader, leave=False, desc="Validation Batches"):
    output = loaded_model(input_batch.to(get_device())).to(get_device())
    flattened_output.append(output[batch_mask].reshape(-1, output.shape[-1]))

  completely_flattened_output = []
  for batch in flattened_output:
    for a in batch:
      completely_flattened_output.append(a)

  prediction_indices = index
  prediction_labels = []




  category_map2 = {0: "O",
                1: "F-ADJ",
                2:"F-PRON", 
                3: "F-DET", 
                4: "M-ADJ",
                5: "M-PRON", 
                6: "M-DET",
                7: "NOUN"}

  for i in range(len(completely_flattened_output)):
    max_val_index = int(torch.max(completely_flattened_output[i], 0).indices)
  

    prediction_labels.append(category_map2.get(max_val_index))

  output = []
  for sentence in Set: 
    for word in sentence:
      output.append(adjectiveGender(word))
  return prediction_labels






In [ ]:
#if word is adjective and it is described as F-adj, make female 
#if word is adjective and F-Noun is close, also make adjective 
textres1 = "La paloma pasó por la calle un poco nerviosa. Voló por lo enojada que estaba."
textres2 = "El número es alto, pero no inesperado."
textres3 = "Ella dice que debes prepararte para lo inesperado."
textres4 = "Es muy inesperado que ellos toquen esa canción."
textres5 = "Un nivel de dificultad inesperado para los chicos."
textres6 = "Fue un poco inesperado, pero no del todo desagradable."
textres7 = "Un nivel de dificultad inesperado para los chicos."
textres8 = "Espera lo inesperado, eso es todo lo que estoy diciendo."
textres9 = "Ojalá hubiera tenido tiempo para empacar, pero mi viaje fue bastante inesperado."
textres10 = "Un cambio inesperado y la destrucción resultarán en tu liberación."
textres11 = "Para Leo, ser despedido fue un shock inesperado."
textres12 = "Tenemos que conseguir el equipo de Yellowknife en tiempo para configurar."
textres13 = "El paso final es configurar el nombre del trabajo."
textres14 = "El primer argumento es el dispositivo a configurar, en este caso /dev/ccd0c."
textres15 = "Esperar 10 a 15 minutos para que pueda configurar un poco."
textres16 = "Siga los siguientes pasos para configurar la conexión de la impresora."
textres17 = "Si no lo tienes, debe ser fácil de configurar."
textres18 = "La Consola de Administración se puede configurar en dos idiomas: Inglés o Español."
textres19 = "El, de hecho, el otro dia me envió estos gemelos."
textres20 = "Puedes ir a buscar más gemelos si quieres."




print(predict(textres1))
print(predict(textres2))
print(predict(textres3))
print(predict(textres4))
print(predict(textres5))
print(predict(textres6))
print(predict(textres7))
print(predict(textres8))
print(predict(textres9))
print(predict(textres10))
print(predict(textres11))
print(predict(textres12))
print(predict(textres13))
print(predict(textres14))
print(predict(textres15))
print(predict(textres16))
print(predict(textres17))
print(predict(textres18))
print(predict(textres19))
print(predict(textres20))








Validation Batches:   0%|          | 0/1 [00:00<?, ?it/s]

['F-DET', 'O', 'O', 'O', 'F-DET', 'NOUN', 'M-DET', 'O', 'M-ADJ', 'O', 'O', 'M-PRON', 'M-ADJ', 'O', 'O']


Validation Batches:   0%|          | 0/1 [00:00<?, ?it/s]

['M-DET', 'NOUN', 'O', 'M-ADJ', 'O', 'O', 'O', 'M-ADJ']


Validation Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

['F-DET', 'O', 'O', 'O', 'M-PRON', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


Validation Batches:   0%|          | 0/1 [00:00<?, ?it/s]

['M-DET', 'O', 'O', 'M-ADJ', 'O', 'M-DET', 'M-PRON', 'O', 'M-PRON', 'O', 'M-DET', 'O']


Validation Batches:   0%|          | 0/1 [00:00<?, ?it/s]

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:


def translateCorrect(englishSentence):
  spanishSentence = translate(englishSentence)
  spanishTok = tokenize(spanishSentence)
  print(spanishTok)
  labels = predict(spanishSentence)
  for index in range(len(labels)):
    if(labels[index] == 'F-ADJ'):
      if(spanishTok[index][-1] == 's'):
        spanishTok[index] = spanishTok[index][0:-2] + "as"
      else:
        spanishTok[index] = spanishTok[index][0:-1] + "a"
  return " ".join(spanishTok)




In [ ]:
test = "The girl was terrified of going outside. She thinks she will become ugly from exposure to the sun. Beauty is temporary. It is beautiful but it fades away at the end."
translateCorrect(test)

['La', 'niña', 'estaba', 'aterrorizada', 'de', 'salir', '.', 'Ella', 'piensa', 'que', 'se', 'volverá', 'fea', 'por', 'la', 'exposición', 'al', 'sol', '.', 'La', 'belleza', 'es', 'temporal', '.', 'Es', 'hermoso', 'pero', 'se', 'desvanece', 'al', 'final']


Validation Batches:   0%|          | 0/1 [00:00<?, ?it/s]

'La niña estaba aterrorizada de salir . Ella piensa que se volverá fea por la exposición al sol . La belleza es temporal . Es hermoso pero se desvanece al final'